In [2]:
#following libraries to establish connection between user and ssh remote server and also to access sql in server.
import sshtunnel
import mysql.connector
import pandas as pd
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import sys
from pathlib import Path

In [7]:
home = expanduser('~')
# mypkey = paramiko.RSAKey.from_private_key_file(home + pkeyfilepath)
# if you want to use ssh password use - ssh_password='your ssh password', bellow
#below are remote ssh & sql variables. 
sql_hostname = '127.0.0.1'
sql_username = 'ifraz-re'
sql_password = '1q2w3e4r'
sql_main_database = 'ghtorrent_201906'
sql_port = 3306
ssh_host = '163.221.172.90'
ssh_user = 'ifraz-re'
ssh_port = 22
ssh_pwd = 'dangerous123'
# below ssh connection code

with SSHTunnelForwarder(
       (ssh_host, ssh_port),
       ssh_username=ssh_user,
       ssh_password=ssh_pwd,
       remote_bind_address=(sql_hostname, sql_port)) as tunnel:
   
 # below open connection with sql @ server thru ssh. 
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
        passwd=sql_password, db=sql_main_database,
        port=tunnel.local_bind_port)
 # below loading attribute from local csv file.
    file1 = pd.read_csv("gh_login_id_name.csv")
    login_data=file1['login_id']
# below limit the for loop for specific iterations & query to match column between table @ remote server sql and local csv file in a local machine.     

    #limit=500
    j=0   
    return_com_comments=[]
    for i in login_data:
        sys.stdout.write("\rExtracting data {} / {} --> {}".format(j+1, len(login_data), i))
        sys.stdout.flush()
        temp1=[]
        query = ("SELECT A.user_id, B.project_id, B.id as commits_id, A.id as commits_comments_id FROM commit_comments A INNER JOIN commits B ON A.commit_id = B.id WHERE A.user_id='%s'" % i)
        try:
            data = (pd.read_sql_query(query, conn)).values.tolist()
            for n in data:
                temp1.append(n)
            return_com_comments = return_com_comments + temp1
        except:
            pass
        
        j+=1
        #if j == limit:
        #    break
     
    conn.close()
print("\nFINISH")

Extracting data 12970 / 12970 --> 32440943
FINISH


In [8]:
return_com_comments

[[37740533, 125728186, 1296026026, 6185214],
 [44008609, 113257935, 1167189850, 5944257],
 [51051921, 134279979, 1380678448, 6345917],
 [51051921, 134279979, 1380678448, 6345918],
 [40298430, 98462684, 1078689043, 5743122],
 [190017, 24449539, 336240062, 3182686],
 [190017, 24449539, 481900775, 3978508],
 [190017, 24449539, 597521641, 4385962],
 [13232234, 56272987, 616563881, 4455735],
 [43571134, 97865033, 1008413234, 5583185],
 [43571134, 97865033, 1008413234, 5583186],
 [43571134, 97865033, 1008413234, 5583187],
 [6106713, 13402741, 184201372, 1969889],
 [42431291, 108288964, 1116630390, 5823281],
 [9395850, 92092690, 963627579, 5696568],
 [2663363, 59503381, 678847093, 4658608],
 [30595985, 127637897, 1319611416, 6226190],
 [46573521, 125697770, 1295772980, 6184577],
 [46573521, 125697770, 1295875295, 6184834],
 [46573521, 125697770, 1295879936, 6184867],
 [46573521, 125697770, 1295897928, 6184922],
 [46573521, 125697770, 1296952900, 6186937],
 [46573521, 125697770, 1297096914, 61

In [9]:
mydf= pd.DataFrame(return_com_comments,columns=['owner_id','project_id','commits_id','commits_comments_id'])
mydf.to_csv("commit_comments_projects.csv", index = False)
mydf

,owner_id,project_id,commits_id,commits_comments_id
0,37740533,125728186,1296026026,6185214
1,44008609,113257935,1167189850,5944257
2,51051921,134279979,1380678448,6345917
3,51051921,134279979,1380678448,6345918
4,40298430,98462684,1078689043,5743122
5,190017,24449539,336240062,3182686
6,190017,24449539,481900775,3978508
7,190017,24449539,597521641,4385962
8,13232234,56272987,616563881,4455735
9,43571134,97865033,1008413234,5583185
